In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

import random 

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score

from datetime import datetime

In [2]:
bp = pd.read_csv('Data/bp_hypnos.csv')
bp['DATE.TIME'] = bp['DATE.TIME'].apply(lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S'))

In [3]:
# bp normaal 100 - 135
# bp preventief 135 - 160
# RIP > 160

# bp normaal 60 - 85
# preventief 85 - 100
# RIP > 100

In [6]:
options = {'Real bad': 1, 'Normal': 2, 'All good': 3}

heart_rate = []
steps = []
sleep = []
bp_onder = []
bp_boven = []

for x in range(100000):
    heart_rate.append(random.choice(list(options.keys())))
    steps.append(random.randint(0, 10000))
    sleep.append(random.randint(5, 8))
    bp_onder.append(random.randint(50, 120))
    bp_boven.append(random.randint(80, 200))
    
def verdict(df):
    
    # mean and std
    steps_mean = df.Steps.mean()
    steps_std = df.Steps.std()
    
    sleep_mean = df.Sleep.mean()
    sleep_std = df.Sleep.std()
    
    verdict = []

    for index, row in df.iterrows():
        count = 0
        # heart rate
        count += options[row.HR]

        # steps
        if row.Steps < (steps_mean - steps_std):
            count += 1
        elif row.Steps > (steps_mean + steps_std):
            count += 3
        else:
            count += 2

        # sleep
        if row.Sleep < (sleep_mean - sleep_std):
            count += 1
        elif row.Sleep > (sleep_mean + sleep_std):
            count += 3
        else: 
            count += 2
            
        # bp boven
        if row.bp_boven < 100: 
            count += 1
        elif row.bp_boven < 135 :
            count += 3
        elif row.bp_boven > 135 and row.bp_boven <= 160:
            count += 2
        elif row.bp_boven > 160: 
            count += 1
            
        # bp onder
        if row.bp_onder < 60:
            count += 1
        elif row.bp_onder < 85:
            count += 3
        elif row.bp_onder > 85 and row.bp_onder <= 100:
            count += 2
        elif row.bp_onder > 100:
            count += 1

        # Hoe heb ik dit ook alweer bedacht, als je overal 1 invult, dan kom je nu uit op 5
        if count < 10:
            verdict.append(1)
        elif count > 12:
            verdict.append(3)
        else:
            verdict.append(2)

        # reset
        count = 0
        
    return verdict

In [7]:
df = pd.DataFrame(data = {'HR': heart_rate, 'Steps': steps, 'Sleep': sleep,
                         'bp_onder': bp_onder, 'bp_boven': bp_boven})

# 1 is real bad; 2 is normal; 3 is real good
df['Score'] = verdict(df)

# one hot encoding
dummy = pd.get_dummies(df['HR'], prefix = 'HR')
new_df = pd.merge(left = df, right = dummy, left_index = True, right_index = True)
new_df

,HR,Steps,Sleep,bp_onder,bp_boven,Score,HR_All good,HR_Normal,HR_Real bad
0,Normal,6908,7,109,97,1,0,1,0
1,Real bad,7178,5,101,99,1,0,0,1
2,All good,5527,8,104,135,1,1,0,0
3,All good,7065,5,69,175,2,1,0,0
4,All good,9460,8,70,92,3,1,0,0
...,...,...,...,...,...,...,...,...,...
99995,Real bad,8483,6,74,122,2,0,0,1
99996,Real bad,2278,7,89,116,2,0,0,1
99997,Real bad,8205,8,99,187,2,0,0,1
99998,Normal,9305,8,87,171,2,0,1,0


In [18]:
features = ['Steps', 'Sleep', 'HR_All good', 'HR_Normal', 'HR_Real bad', 'bp_onder', 'bp_boven']

X = new_df[features]
y = new_df.Score

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3, random_state = 42)

In [19]:
model = DecisionTreeClassifier(max_depth = 8) # anders returned ie altijd 100% --> overfitting

# scaler = MinMaxScaler()
# scaler.fit_transform(x_train)



model = model.fit(x_train, y_train)

pred = model.predict(x_test)

print("Accuracy: ", round(accuracy_score(y_test, pred) * 100, 1), '%')

Accuracy:  89.5 %


In [20]:
new_df.Score.value_counts() # ff checken wat de verdeling is

2    47853
1    46443
3     5704
Name: Score, dtype: int64

In [21]:
text_representation = tree.export_text(model, feature_names = features)

for feature in features:
    print(feature, text_representation.count(feature))

# print(text_representation)

Steps 62
Sleep 34
HR_All good 20
HR_Normal 18
HR_Real bad 2
bp_onder 52
bp_boven 90


In [22]:
fig = plt.figure(figsize = (25, 20))

features = ['Steps', 'Sleep', 'HR_All good', 'HR_Normal', 'HR_Real bad', 'bp_onder', 'bp_boven']
target = y.unique().astype(str)

_ = tree.plot_tree(model, feature_names = features, class_names = target, filled = True)

plt.savefig('guntoekomst')

Error in callback <function flush_figures at 0x11ecd1550> (for post_execute):


KeyboardInterrupt: 

In [ ]:
import graphviz # zelfde als hierboven, maar dan fancier ofzo

In [ ]:
dot_data = tree.export_graphviz(model, out_file = None, feature_names = features, 
                                class_names = target, filled = True)

graph = graphviz.Source(dot_data, format = 'png')


In [ ]:
new_df

In [ ]:
def test(steps, sleep, hr_good, hr_normal, hr_bad, bp_onder, bp_boven)

    model.predict([[sleep, hr_good, hr

In [29]:
def test(steps, sleep, hr_good, hr_normal, hr_bad, bp_onder, bp_boven):

    return (model.predict([[steps, sleep, hr_good, hr_normal, hr_bad, bp_onder, bp_boven]]))[0] 

In [30]:
test(6000, 6, 0, 1, 0, 80, 140)

/opt/homebrew/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


2

In [35]:
model.decision_path([[6000, 6, 0, 1, 0, 80, 140]])

  (0, 0)	1
  (0, 1)	1
  (0, 67)	1
  (0, 68)	1
  (0, 96)	1
  (0, 112)	1
  (0, 120)	1
  (0, 121)	1
  (0, 122)	1


/opt/homebrew/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [48]:
feature = model.tree_.feature
threshold = model.tree_.threshold

X_test = [[6000, 6, 0, 1, 0, 80, 140]]

node_indicator = model.decision_path(X_test)
leaf_id = model.apply(X_test)

sample_id = 0
# obtain ids of the nodes `sample_id` goes through, i.e., row `sample_id`
node_index = node_indicator.indices[
    node_indicator.indptr[sample_id] : node_indicator.indptr[sample_id + 1]
]

print("Rules used to predict sample {id}:\n".format(id=sample_id))
for node_id in node_index:
    
    # continue to the next node if it is a leaf node
    if leaf_id[sample_id] == node_id:
        continue

    # check if value of the split feature for sample 0 is below threshold
    if X_test[sample_id][feature[node_id]] <= threshold[node_id]:
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print(
        "decision node {node} : (X_test[{sample}, {feature}] = {value}) "
        "{inequality} {threshold})".format(
            node=node_id,
            sample=sample_id,
            feature=feature[node_id],
            value=X_test[sample_id][feature[node_id]],
            inequality=threshold_sign,
            threshold=threshold[node_id],
        )
    )

Rules used to predict sample 0:

decision node 0 : (X_test[0, 4] = 0) <= 0.5)
decision node 1 : (X_test[0, 1] = 6) > 5.5)
decision node 67 : (X_test[0, 5] = 80) <= 100.5)
decision node 68 : (X_test[0, 5] = 80) > 59.5)
decision node 96 : (X_test[0, 6] = 140) > 134.5)
decision node 112 : (X_test[0, 0] = 6000) > 2084.5)
decision node 120 : (X_test[0, 5] = 80) <= 84.5)
decision node 121 : (X_test[0, 0] = 6000) <= 7874.5)


/opt/homebrew/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
